In [16]:
from __future__ import division
import numpy
import re
import sys
import scipy.misc
import imageio
import vifp
import psnr
import reco
import ssim





In [17]:
# ! sudo apt install -y ffmpeg
! sudo apt-get install -y p7zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.


In [18]:
! ffmpeg -version

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq 

In [10]:
! ffmpeg -codecs

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [6]:
! ls

MeasureNotebook.ipynb  jpg_demo_quality.png  measure.py  ssim_theano.py
QoS.ipynb	       jpg_demo_size.png     psnr.py	 vifp.py
__pycache__	       lena.png		     reco.py	 waterfall_cif.7z
jpg_demo.py	       lena_tmp.jpg	     ssim.py	 waterfall_cif.yuv


In [21]:
! ffprobe -v quiet -print_format json -show_format -show_streams ./input.mkv

{
    "streams": [
        {
            "index": 0,
            "codec_name": "h264",
            "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10",
            "profile": "Main",
            "codec_type": "video",
            "codec_time_base": "1/50",
            "codec_tag_string": "[0][0][0][0]",
            "codec_tag": "0x0000",
            "width": 1280,
            "height": 720,
            "coded_width": 1280,
            "coded_height": 720,
            "has_b_frames": 1,
            "sample_aspect_ratio": "0:1",
            "display_aspect_ratio": "0:1",
            "pix_fmt": "yuv420p",
            "level": 31,
            "chroma_location": "left",
            "field_order": "progressive",
            "refs": 1,
            "is_avc": "true",
            "nal_length_size": "4",
            "r_frame_rate": "25/1",
            "avg_frame_rate": "25/1",
            "time_base": "1/1000",
            "start_pts": 0,
            "start_time": "0.000000",
         

In [9]:
! sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
! sudo chmod a+rx /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     3    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   632  100   632    0     0    325      0  0:00:01  0:00:01 --:--:--   325
100 1740k  100 1740k    0     0   448k      0  0:00:03  0:00:03 --:--:-- 1121k


In [28]:
# ! youtube-dl "https://www.youtube.com/watch?v=w01V5FI03MQ"
! ls
# ! ffmpeg -i input1280x720.mkv -c:v rawvideo -pix_fmt yuv420p input1280x720.yuv
# ! rm 'Hidden Universe in 30 seconds-w01V5FI03MQ.mkv'

MeasureNotebook.ipynb  input1280x720.mkv     lena_tmp.jpg  ssim_theano.py
QoS.ipynb	       input1280x720.yuv     measure.py    vifp.py
__pycache__	       jpg_demo.py	     output.avi    waterfall_cif.7z
hiddenUni.avi	       jpg_demo_quality.png  psnr.py	   waterfall_cif.yuv
hiddenUni.mp4	       jpg_demo_size.png     reco.py
hiddenUni.webm	       lena.png		     ssim.py


In [20]:
# ref_file = "waterfall_cif.yuv"
# ! ffmpeg -i $ref_file output.webm
# ! ffmpeg -i $ref_file -c:v vp9 -c:a libvorbis output.mkv
# ! ffmpeg -i $ref_file -vcodec libx264 -acodec aac output.mp4
# ! ffmpeg -i input1280x720.mkv hiddenUni.avi
! ffmpeg -y -v -10 -i "input1280x720.mkv" -an -sn -threads 8 -vsync 0 -r 25000/1000 -pix_fmt yuv420p -f rawvideo - | x265 --input - --input-res 640x352 --fps 25 --frames 429 --output "H:\Temp\test_19_53_58_0110_01.26
#         FLV

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [4]:

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python 2/3 compatibility
from __future__ import print_function
from __future__ import division
import re
import sys
import scipy.misc
import imageio
import vifp
import psnr
import reco
import ssim
import numpy as np
import cv2 as cv
import argparse


In [3]:
def img_greyscale(img):
    return 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]

ref_file = "hiddenUni.mp4"
dest_file = "hiddenUni.mkv"

sourceReference = ref_file
sourceCompareWith = dest_file
delay = 30
psnrTriggerValue = 30
framenum = -1 # Frame counter
captRefrnc = cv.VideoCapture(sourceReference)
captUndTst = cv.VideoCapture(sourceCompareWith)
if not captRefrnc.isOpened():
    print("Could not open the reference " + sourceReference)
    sys.exit(-1)
if not captUndTst.isOpened():
    print("Could not open case test " + sourceCompareWith)
    sys.exit(-1)
refS = (int(captRefrnc.get(cv.CAP_PROP_FRAME_WIDTH)), int(captRefrnc.get(cv.CAP_PROP_FRAME_HEIGHT)))
uTSi = (int(captUndTst.get(cv.CAP_PROP_FRAME_WIDTH)), int(captUndTst.get(cv.CAP_PROP_FRAME_HEIGHT)))
if refS != uTSi:
    print("Inputs have different size!!! Closing.")
    sys.exit(-1)
print("Reference frame resolution: Width={} Height={} of Number of Frames: {}".format(refS[0], refS[1],
                                                                             captRefrnc.get(cv.CAP_PROP_FRAME_COUNT)))
print("PSNR trigger value {}".format(psnrTriggerValue))
while framenum < 5: # Show the image captured in the window and repeat
         
    _, frameReference = captRefrnc.read()
    _, frameUnderTest = captUndTst.read()
    frameReference = cv.cvtColor(frameReference, cv.COLOR_BGR2GRAY)
    frameUnderTest = cv.cvtColor(frameUnderTest, cv.COLOR_BGR2GRAY)
    ref = frameReference
    dist = frameUnderTest
    if frameReference is None or frameUnderTest is None:
        break
    framenum += 1
    print("\nFrame: {}".format(framenum))
    vifp_value = vifp.vifp_mscale(ref, dist)
    print("VIFP={}".format(vifp_value), end=" ")

    ssim_value = ssim.ssim_exact(ref/255, dist/255)
    print("SSIM={}".format(ssim_value), end=" ")

    psnr_value = psnr.psnr(ref, dist)
    print("PSNR={}".format(psnr_value), end=" ") 

    reco_value = reco.reco(ref/255, dist/255)
    print("RECO={}".format(reco_value), end=" ") 
    k = cv.waitKey(delay)
    if k == 27:
        break
sys.exit(0)

Reference frame resolution: Width=1280 Height=720 of Number of Frames: 938.0
PSNR trigger value 30

Frame: 0
VIFP=0.9999999999860117 SSIM=1.0 PSNR=100 

KeyboardInterrupt: 

In [10]:
! ls

MeasureNotebook.ipynb  input1280x720.yuv     o.json	 ssim_theano.py
QoS.ipynb	       jpg_demo.py	     out.json	 vifp.py
__pycache__	       jpg_demo_quality.png  output.avi  waterfall_cif.7z
hiddenUni.avi	       jpg_demo_size.png     output.y4m  waterfall_cif.yuv
hiddenUni.mkv	       lena.png		     psnr.py
hiddenUni.mp4	       lena_tmp.jpg	     reco.py
hiddenUni.webm	       measure.py	     ssim.py


In [ ]:
    ref_file = "hiddenUni.mp4"
    dest_file = "hiddenUni.mkv"
    sourceReference = ref_file
    sourceCompareWith = dest_file
    delay = 30
    psnrTriggerValue = 30
    framenum = -1 # Frame counter
    captRefrnc = cv.VideoCapture(sourceReference)
    captUndTst = cv.VideoCapture(sourceCompareWith)
    if not captRefrnc.isOpened():
        print("Could not open the reference " + sourceReference)
        sys.exit(-1)
    if not captUndTst.isOpened():
        print("Could not open case test " + sourceCompareWith)
        sys.exit(-1)
    refS = (int(captRefrnc.get(cv.CAP_PROP_FRAME_WIDTH)), int(captRefrnc.get(cv.CAP_PROP_FRAME_HEIGHT)))
    uTSi = (int(captUndTst.get(cv.CAP_PROP_FRAME_WIDTH)), int(captUndTst.get(cv.CAP_PROP_FRAME_HEIGHT)))
    if refS != uTSi:
        print("Inputs have different size!!! Closing.")
        sys.exit(-1)
    print("Reference frame resolution: Width={} Height={} of Number of Frames: {}".format(refS[0], refS[1],
                                                                             captRefrnc.get(cv.CAP_PROP_FRAME_COUNT)))
    print("PSNR trigger value {}".format(psnrTriggerValue))
    while framenum < 200: # Show the image captured in the window and repeat
         
        _, frameReference = captRefrnc.read()
        _, frameUnderTest = captUndTst.read()
        frameReference = cv.cvtColor(frameReference, cv.COLOR_BGR2GRAY)
        frameUnderTest = cv.cvtColor(frameUnderTest, cv.COLOR_BGR2GRAY)
        ref = frameReference
        dist = frameUnderTest
        if framenum > 5 and frameReference is None or frameUnderTest is None:
            break
        framenum += 1
        print("\nFrame: {}".format(framenum))
        vifp_value = vifp.vifp_mscale(ref, dist)
        print("VIFP={}".format(vifp_value), end=" ")

        ssim_value = ssim.ssim_exact(ref/255, dist/255)
        print("SSIM={}".format(ssim_value), end=" ")

        psnr_value = psnr.psnr(ref, dist)
        print("PSNR={}".format(psnr_value), end=" ") 

        reco_value = reco.reco(ref/255, dist/255)
        print("RECO={}".format(reco_value), end=" ") 
        k = cv.waitKey(delay)
        if k == 27:
            break
    sys.exit(0)


Reference frame resolution: Width=1280 Height=720 of Number of Frames: 938.0
PSNR trigger value 30

Frame: 0
VIFP=0.9999999999860117 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 1
VIFP=0.9999999999860006 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 2
VIFP=0.9999999999860706 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 3
VIFP=0.9999999999860998 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 4
VIFP=0.9999999999861032 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 5
VIFP=0.9999999999861428 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 6
VIFP=0.9999999999859022 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 7
VIFP=0.999999999985875 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 8
VIFP=0.9999999999858178 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 9
VIFP=0.9999999999858139 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 10
VIFP=0.999999999985813 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 11
VIFP=0.9999999999858084 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 12
VIFP=0.9999999999858128 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 13
VIFP=0.999999999985819 SSIM=1.0 PSNR=100 RECO=1.0 
Frame: 14
VIFP=0.9999999999858198 SSIM=1.0 PS

In [69]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python 2/3 compatibility
from __future__ import print_function
import numpy as np
import cv2 as cv
import argparse
import sys

def img_greyscale(img):
    return 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]

def getPSNR(I1, I2):
    s1 = cv.absdiff(I1, I2) #|I1 - I2|
    s1 = np.float32(s1)     # cannot make a square on 8 bits
    s1 = s1 * s1            # |I1 - I2|^2
    sse = s1.sum()          # sum elements per channel
    if sse <= 1e-10:        # sum channels
        return 0            # for small values return zero
    else:
        shape = I1.shape
        mse = 1.0 * sse / (shape[0] * shape[1] * shape[2])
        psnr = 10.0 * np.log10((255 * 255) / mse)
        return psnr

    
def getMSSISM(i1, i2):
    C1 = 6.5025
    C2 = 58.5225
    # INITS
    I1 = np.float32(i1) # cannot calculate on one byte large values
    I2 = np.float32(i2)
    I2_2 = I2 * I2 # I2^2
    I1_2 = I1 * I1 # I1^2
    I1_I2 = I1 * I2 # I1 * I2
    # END INITS
    # PRELIMINARY COMPUTING
    mu1 = cv.GaussianBlur(I1, (11, 11), 1.5)
    mu2 = cv.GaussianBlur(I2, (11, 11), 1.5)
    mu1_2 = mu1 * mu1
    mu2_2 = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_2 = cv.GaussianBlur(I1_2, (11, 11), 1.5)
    sigma1_2 -= mu1_2
    sigma2_2 = cv.GaussianBlur(I2_2, (11, 11), 1.5)
    sigma2_2 -= mu2_2
    sigma12 = cv.GaussianBlur(I1_I2, (11, 11), 1.5)
    sigma12 -= mu1_mu2
    t1 = 2 * mu1_mu2 + C1
    t2 = 2 * sigma12 + C2
    t3 = t1 * t2                    # t3 = ((2*mu1_mu2 + C1).*(2*sigma12 + C2))
    t1 = mu1_2 + mu2_2 + C1
    t2 = sigma1_2 + sigma2_2 + C2
    t1 = t1 * t2                    # t1 =((mu1_2 + mu2_2 + C1).*(sigma1_2 + sigma2_2 + C2))
    ssim_map = cv.divide(t3, t1)    # ssim_map =  t3./t1;
    mssim = cv.mean(ssim_map)       # mssim = average of ssim map
    return mssim



def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("-d", "--delay", type=int, default=30, help=" Time delay")
    parser.add_argument("-v", "--psnrtriggervalue", type=int, default=30, help="PSNR Trigger Value")
    parser.add_argument("-r", "--ref", type=str, default="Megamind.avi", help="Path to reference video")
    parser.add_argument("-t", "--undertest", type=str, default="Megamind_bugy.avi",
                        help="Path to the video to be tested")
    args = parser.parse_args()
    sourceReference = args.ref
    sourceCompareWith = args.undertest
    delay = args.delay
    psnrTriggerValue = args.psnrtriggervalue
    framenum = -1 # Frame counter
    captRefrnc = cv.VideoCapture(sourceReference)
    captUndTst = cv.VideoCapture(sourceCompareWith)
    if not captRefrnc.isOpened():
        print("Could not open the reference " + sourceReference)
        sys.exit(-1)
    if not captUndTst.isOpened():
        print("Could not open case test " + sourceCompareWith)
        sys.exit(-1)
    refS = (int(captRefrnc.get(cv.CAP_PROP_FRAME_WIDTH)), int(captRefrnc.get(cv.CAP_PROP_FRAME_HEIGHT)))
    uTSi = (int(captUndTst.get(cv.CAP_PROP_FRAME_WIDTH)), int(captUndTst.get(cv.CAP_PROP_FRAME_HEIGHT)))
    if refS != uTSi:
        print("Inputs have different size!!! Closing.")
        sys.exit(-1)
    WIN_UT = "Under Test"
    WIN_RF = "Reference"
    cv.namedWindow(WIN_RF, cv.WINDOW_AUTOSIZE)
    cv.namedWindow(WIN_UT, cv.WINDOW_AUTOSIZE)
    cv.moveWindow(WIN_RF, 400, 0) #750,  2 (bernat =0)
    cv.moveWindow(WIN_UT, refS[0], 0) #1500, 2
    print("Reference frame resolution: Width={} Height={} of nr#: {}".format(refS[0], refS[1],
                                                                             captRefrnc.get(cv.CAP_PROP_FRAME_COUNT)))
    print("PSNR trigger value {}".format(psnrTriggerValue))
    while True: # Show the image captured in the window and repeat
        _, frameReference = captRefrnc.read()
        _, frameUnderTest = captUndTst.read()
        if frameReference is None or frameUnderTest is None:
            print(" < < <  Game over!  > > > ")
            break
        framenum += 1
        psnrv = getPSNR(frameReference, frameUnderTest)
        print("Frame: {}# {}dB".format(framenum, round(psnrv, 3)), end=" ")
        if (psnrv < psnrTriggerValue and psnrv):
            mssimv = getMSSISM(frameReference, frameUnderTest)
            print("MSSISM: R {}% G {}% B {}%".format(round(mssimv[2] * 100, 2), round(mssimv[1] * 100, 2),
                                                     round(mssimv[0] * 100, 2)), end=" ")
        vifp_value = vifp.vifp_mscale(ref, dist)
        print("VIFP={}".format(vifp_value))

        ssim_value = ssim.ssim_exact(ref/255, dist/255)
        print("SSIM={}".format(ssim_value))

        psnr_value = psnr.psnr(ref, dist)
        print("PSNR={}".format(psnr_value)) 

        reco_value = reco.reco(ref/255, dist/255)
        print("RECO={}".format(reco_value)) 
        cv.imshow(WIN_RF, frameReference)
        cv.imshow(WIN_UT, frameUnderTest)
        k = cv.waitKey(delay)
        if k == 27:
            break
    sys.exit(0)
if __name__ == "__main__":
    main()

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [52]:
def img_greyscale(img):
    return 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]

def img_read_yuv(src_file, width, height):
    y_img = numpy.fromfile(src_file, dtype=numpy.uint8, count=(width * height)).reshape( (height, width) )
    u_img = numpy.fromfile(src_file, dtype=numpy.uint8, count=((width/2) * (height/2))).reshape( (height/2, width/2) )
    v_img = numpy.fromfile(src_file, dtype=numpy.uint8, count=((width/2) * (height/2))).reshape( (height/2, width/2) )
    return (y_img, u_img, v_img)

ref_file = "lena.png"
dist_file = "lena_tmp.jpg"

if ".yuv" in ref_file:
    # Inputs are uncompressed video in YUV420 planar format

    # Get resolution from file name
    m = re.search(r"(\d+)x(\d+)", ref_file)
    if not m:
        print("Could not find resolution in file name: {}".format(ref_file))
        exit(1)

    width, height = int(m.group(1)), int(m.group(2))
    print("Comparing {} to {}, resolution {} x {}".format(ref_file, dist_file, width, height))

    ref_fh = open(ref_file, "rb")
    dist_fh = open(dist_file, "rb")

    frame_num = 0
    while True:
        ref, _, _ = img_read_yuv(ref_fh, width, height)
        dist, _, _ = img_read_yuv(dist_fh, width, height)
        vifp_value = vifp.vifp_mscale(ref.astype(float), dist.astype(float))
        ssim_value = ssim.ssim(ref, dist)
        print("Frame={} VIFP={} SSIM={}".format(frame_num, vifp_value, ssim_value))
        frame_num += 1

else:
    # Inputs are image files
    ref = imageio.imread(ref_file, as_gray=True).astype(numpy.float32)
    dist = imageio.imread(dist_file, as_gray=True).astype(numpy.float32)

    width, height = ref.shape[1], ref.shape[0]
    print("Comparing {} to {}, resolution {} x {}".format(ref_file, dist_file, width, height)) 

    vifp_value = vifp.vifp_mscale(ref, dist)
    print("VIFP={}".format(vifp_value))

    ssim_value = ssim.ssim_exact(ref/255, dist/255)
    print("SSIM={}".format(ssim_value))

    psnr_value = psnr.psnr(ref, dist)
    print("PSNR={}".format(psnr_value)) 

    reco_value = reco.reco(ref/255, dist/255)
    print("RECO={}".format(reco_value)) 


Comparing lena.png to lena_tmp.jpg, resolution 512 x 512
VIFP=0.9826524488454484


AttributeError: 'function' object has no attribute 'ssim_exact'